In [ ]:
from keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import KFold
from keras.regularizers import l2
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# TensorFlow e tf.keras
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model

# Bibliotecas Auxiliares
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as sm
from collections import Counter
import gc
from IPython.display import clear_output
from keras.utils import to_categorical

In [ ]:
len(Counter(y_train).keys())

In [ ]:
def split_data(X, y, n_splits, random_state=None):
    # Define a semente do gerador de números aleatórios
    np.random.seed(random_state)

    # Cria uma lista de índices embaralhados
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    # Aplica os índices embaralhados a X e y
    X = X[indices]
    y = y[indices]

    # Calcula os tamanhos dos splits
    split_sizes = np.full(n_splits, len(X) // n_splits)
    split_sizes[:len(X) % n_splits] += 1

    # Divide X e y de acordo com os tamanhos dos splits
    X_splits = np.split(X, np.cumsum(split_sizes[:-1]))
    y_splits = np.split(y, np.cumsum(split_sizes[:-1]))

    # Retorna a lista de subconjuntos de X e y
    return [[X_i, y_i] for X_i, y_i in zip(X_splits, y_splits)]


def create_model(input_shape=(28, 28, 1), qtd_classes=10):
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.3),

        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),

        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        Dropout(0.5),

        Flatten(),

        Dense(512, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Dropout(0.5),

        Dense(qtd_classes, activation='softmax')
    ])
    return model

def train_model(model, X_train, y_train, epochs):
    model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    # Criar modelo

    # Definir callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=5, min_lr=0.001)
    early_stopping = EarlyStopping(monitor='val_accuracy', patience=20)

    # Treinar modelo
    history = model.fit(
        X_train, y_train, 
        epochs=epochs, 
        validation_split=0.2, 
        callbacks=[reduce_lr, early_stopping]
    )
    
    return model, history



def evaluate_model(history, X_test, y_test, model, class_names):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.show()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper right')
    plt.show()

    score = model.evaluate(X_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

    y_pred = model.predict(X_test)

    # Convertendo as previsões de matriz de classe binária para vetor de classificações de classe
    y_pred_classes = np.argmax(y_pred, axis=1)

    print(classification_report(np.argmax(y_test, axis=1), y_pred_classes, target_names=class_names))

    cm = confusion_matrix(np.argmax(y_test, axis=1), y_pred_classes)

    plt.figure(figsize=(8, 6))
    sns.set(font_scale=1.4)
    heatmap = sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', cbar=False)

    heatmap.set_xticklabels(class_names, rotation=45)
    heatmap.set_yticklabels(class_names, rotation=45)

    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()



In [ ]:
# Baixar a base de dados
fashion_mnist = keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()


# Convertendo o vetor de classes para matriz de classes binárias
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Definimos o nome das classes
class_names = ['Camiseta', 'Calça', 'Suéter', 'Vestido', 'Casaco',
               'Sandália', 'Camisa', 'Tênis', 'Bolsa', 'Botas']


In [ ]:
X_train.shape

In [ ]:
X_train.shape

In [ ]:
X_y = split_data(X_train, y_train, n_splits = 4, random_state=43)

In [ ]:
len(X_y)

In [ ]:
X_y[0][0].shape

In [ ]:
X_y[0][1].shape

In [ ]:
X_y[1][0].shape

In [ ]:
X, y = X_y[0]

In [ ]:
model = create_model()
X, y = X_y[0]
model, history = train_model(model, X, y, epochs = 50)
gc.collect()
clear_output(wait=True)
evaluate_model(history, X_test, y_test, model, class_names)

In [ ]:
# Para salvar modelo basta usar:
model.save('modelo_v1.h5')

E se tivermos mais dados para treinar o modelo? ou tivermos que continuar o treinamento após algum tempo?

Podemos continuar o treinamento do modelo...

In [ ]:
# Realizar a leitura do modelo
model = load_model('modelo_v1.h5')

In [ ]:
# Conseguimos utilizar modelo carregado
evaluate_model(history, X_test, y_test, model, class_names)

In [ ]:
X, y = X_y[1]
model, history = train_model(model, X, y, epochs = 50)
gc.collect()
clear_output(wait=True)
evaluate_model(history, X_test, y_test, model, class_names)

In [ ]:
X, y = X_y[2]
model, history = train_model(model, X, y, epochs = 50)
gc.collect()
clear_output(wait=True)
evaluate_model(history, X_test, y_test, model, class_names)

In [ ]:
X, y = X_y[3]
model, history = train_model(model, X, y, epochs = 50)
gc.collect()
clear_output(wait=True)
evaluate_model(history, X_test, y_test, model, class_names)

In [ ]:
# Para salvar modelo basta usar:
model.save('modelo_v2.h5')